In [1]:
cd ..

/home/alpacapaca/infinityWord


In [2]:
from datasets.seq2seq_datasets import Seq2SeqIndexedDataset

In [3]:
from models.encoders.rnn_encoder import GRUEncoder
from models.decoders.rnn_decoder import GRUDecoder
from models.rnn_model import RNNModel

In [4]:
from trainers.seq2seq_trainer import Seq2SeqTrainer, seq2seq_collate_fn

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [6]:
from utils import get_logger

In [7]:
from datasets.base_datasets import SentencesTokenizedDataset

In [8]:
from dictionaries import BaseDictionary

In [9]:
dictionary = BaseDictionary.load('base_dictionary')
dictionary.vocabulary_size

48034

In [10]:
device = torch.device('cuda')

In [11]:
embeddings = nn.Embedding(num_embeddings=dictionary.vocabulary_size, embedding_dim=128)
encoder = GRUEncoder(embeddings, hidden_size=256)
decoder = GRUDecoder(embeddings, hidden_size=256)
model = RNNModel(encoder, decoder)
logger = get_logger(log_name='gru_encoder_decoder')
trainer = Seq2SeqTrainer(model=model, 
                         train_dataloader=DataLoader(Seq2SeqIndexedDataset('train'), batch_size=32, collate_fn=seq2seq_collate_fn), 
                         val_dataloader=DataLoader(Seq2SeqIndexedDataset('val'), batch_size=32, collate_fn=seq2seq_collate_fn),
                         loss_function=CrossEntropyLoss(size_average=False),
                         optimizer=Adam(model.parameters()),
                         device=device,
                         logger=logger)

In [12]:
trainer.run(epochs=10)

  0%|          | 0/817 [00:00<?, ?it/s]


[INFO] 07-30 18:58:21 > Epoch: 0   Progress: 0.0% (0:01:32.889739) Train Loss: 10.6721 Val Loss: 5.37086 Train Perplexity: 4.31e+04 Val Perplexity: 2.15e+02 Learning rate: 0.001 
[INFO] 07-30 18:58:21 > Saving the model...


100%|██████████| 817/817 [02:18<00:00,  5.91it/s]


[INFO] 07-30 19:02:11 > Epoch: 1   Progress: 10.0% (0:05:22.630113) Train Loss: 3.97414 Val Loss: 4.02294 Train Perplexity: 53.2 Val Perplexity: 55.9 Learning rate: 0.001 


 50%|████▉     | 406/817 [00:52<00:53,  7.71it/s]

...

100%|██████████| 817/817 [02:20<00:00,  5.80it/s]


[INFO] 07-30 19:21:19 > Epoch: 6   Progress: 60.0% (0:24:30.169223) Train Loss: 2.905 Val Loss: 4.66593 Train Perplexity: 18.3 Val Perplexity: 1.06e+02 Learning rate: 0.001 


100%|██████████| 817/817 [02:09<00:00,  6.32it/s]


[INFO] 07-30 19:25:00 > Epoch: 7   Progress: 70.0% (0:28:11.274724) Train Loss: 2.69608 Val Loss: 4.92483 Train Perplexity: 14.8 Val Perplexity: 1.38e+02 Learning rate: 0.001 


 10%|█         | 83/817 [00:05<00:52, 13.98it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
import torch

In [14]:
from torch.nn.functional import softmax

In [15]:
from dictionaries import BaseDictionary

In [16]:
dictionary = BaseDictionary.load('base_dictionary')

In [17]:
inputs = torch.tensor([dictionary['<StartSent>']]).unsqueeze(0)
inputs

tensor([[ 2]])

In [18]:
model.eval()

LSTM(
  (embedding): Embedding(50000, 128, padding_idx=0)
  (lstm): LSTM(128, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=50000, bias=True)
)

In [19]:
dictionary.idx2word[softmax(model(inputs), dim=2).argmax().item()]

'탄탄대로의'

In [20]:
from generators import OneSentenceGenerator

In [21]:
generator = OneSentenceGenerator(model, dictionary, checkpoint_filepath='parameters/LSTM/LSTM_0_2018-07-28 19:31:26.237748.pth')

In [23]:
generator.generate_one()

'머리에는 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'